In [1]:
import mysql.connector

# INFO
**На Flask не все запросы реализованы, все запросы представлены в этой тетрадке. Приложение на Flask - пример, как может это выглядеть.**

## Подготовка:
Создание базы данных, таблиц

```python
mydb = mysql.connector.connect(
  host="localhost",
  user="irene",
  password="Vasya123"
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE schedule")
```

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="irene",
  password="Vasya123",
  database="schedule"
)
mycursor = mydb.cursor()

My system's databases

In [3]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('schedule',)
('sys',)


### Задание:
    У института есть преподаватели, учебные группы и аудитории. Организовать хранение расписания.
### Таблицы:
    1) BATCHES (id, Name): type(Name) = TEXT
    2) CLASSES (id, Name): type(Name) = TEXT
    3) BUILDINGS (id, Address): type(Address) = TEXT
    4) CLASSROOMS (id, Number, id Address): type(Number) = INT
    5) STUDENTS (id, Name, id Batch, Email): type(Name) = TEXT, type(Email) = TEXT
    6) SCHEDULE (id, Date_Time, Teacher, id Batch, Classroom, id Building, id Class, id Student): type(Date_Time) = DATETIME

In [4]:
mycursor.execute("DROP TABLE BATCHES, CLASSES, BUILDINGS, CLASSROOMS, STUDENTS, SCHEDULE")

In [5]:
mycursor.execute("CREATE TABLE BATCHES (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT)")
mycursor.execute("CREATE TABLE CLASSES (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT)")
mycursor.execute("CREATE TABLE BUILDINGS (id INT AUTO_INCREMENT PRIMARY KEY, Address TEXT)")
mycursor.execute("CREATE TABLE CLASSROOMS (id INT AUTO_INCREMENT PRIMARY KEY, Number INT, Address INT)")
mycursor.execute("CREATE TABLE STUDENTS (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT, Batch INT, Email TEXT)")
mycursor.execute("CREATE TABLE SCHEDULE (id INT AUTO_INCREMENT PRIMARY KEY, Date_Time DATETIME, Teacher TEXT, \
                    Batch INT, Classroom INT, Building INT, Class INT, Student INT)")

In [6]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('BATCHES',)
('BUILDINGS',)
('CLASSES',)
('CLASSROOMS',)
('SCHEDULE',)
('STUDENTS',)


### Что может делать юзер:
    Вписывать расписание для одного студента
    Вписывать расписание для группы

### Переменные (вводятся юзером):
    teacher_name, batch_name, class_name, building_address, classroom_num, student_name, date_time

In [7]:
def insert(mycursor, mydb, teacher_name, batch_name, class_name, building_address, classroom_num, student_name, date_time, email):
    
    '''
    Вставляем в таблицы, где нет повторов и ссылок.
    '''
    insert_without_dupl = "INSERT INTO %s (%s) SELECT * FROM (SELECT '%s') AS tmp \
                            WHERE NOT EXISTS (SELECT %s FROM %s WHERE %s = '%s')"
    sql = insert_without_dupl % ('BATCHES', 'Name', batch_name, 'Name',
                                 'BATCHES', 'Name', batch_name)
    mycursor.execute(sql)
    sql = insert_without_dupl % ('CLASSES', 'Name', class_name, 'Name',
                                 'CLASSES', 'Name', class_name)
    mycursor.execute(sql)
    sql = insert_without_dupl % ('BUILDINGS', 'Address', building_address,
                                 'Address', 'BUILDINGS', 'Address', building_address)
    mycursor.execute(sql)
    
    '''
    Вставляем в таблицу, где есть повторы и ссылки (имена и номера аудиторий (в двух корпусах есть аудитория 504) могут повторяться).
    '''    
    sql = "INSERT CLASSROOMS (Number, Address) \
            SELECT %s, (SELECT id from BUILDINGS WHERE Address = '%s') \
            WHERE NOT EXISTS \
                (SELECT Number FROM CLASSROOMS WHERE CLASSROOMS.Number = %s \
                    AND CLASSROOMS.Address = (SELECT id from BUILDINGS WHERE Address = '%s'))" % (classroom_num, building_address,
                                                                                                  classroom_num, building_address)
    mycursor.execute(sql)
    
    sql = "INSERT INTO STUDENTS (Name, Batch, Email) \
            SELECT * FROM (SELECT '%s', (SELECT id from BATCHES WHERE Name = '%s'), '%s') AS tmp \
            WHERE NOT EXISTS (SELECT (Email) FROM STUDENTS WHERE Email = '%s')" % (student_name, batch_name,
                                                                                   email, email)
    mycursor.execute(sql)

    sql = "INSERT INTO SCHEDULE (Date_Time, Teacher, Batch, Classroom, Building, Class, Student) \
            VALUES ('%s', '%s', (SELECT id from BATCHES WHERE Name = '%s'), %s, \
                    (SELECT id from BUILDINGS WHERE Address = '%s'), \
                    (SELECT id from CLASSES WHERE Name = '%s'), \
                    (SELECT id from STUDENTS WHERE Email = '%s' \
                    AND Batch = (SELECT id from BATCHES WHERE Name = '%s')))" % (date_time, teacher_name,
                                                                                 batch_name, classroom_num,

                                                                                 building_address, class_name,
                                                                                 email, batch_name)
    mycursor.execute(sql)
    mydb.commit()
    print("Inserted.")

In [8]:
teachers = ['Толдова Светлана Юрьевна', 'Нефедов Михаил Анатольевич',
            'Золотухин Всеволод Валерьевич', 'Золотухин Всеволод Валерьевич',
            'Клышинский Эдуард Станиславович', 'Литвинов Денис Владимирович', 
            'Иванов Иван Иванович']
batches = ['БКЛ-152', 'БКЛ-152', 'БКЛ-152', 'БКЛ-153', 'БКЛ-153', 'БКЛ-153', 'БКЛ-153']
building_address = ['Старая Басманная ул., д. 21/4', 'Кирпичная ул., д. 33',
                    'Старая Басманная ул., д. 21/4', 'Старая Басманная ул., д. 21/4',
                    'Старая Басманная ул., д. 21/4', 'Старая Басманная ул., д. 21/4', 
                    'Старая Басманная ул., д. 21/4']
classrooms = [505, 505, 503, 502, 321, 505, 523]
students = ['Пантелеева Ирина Максимовна', 'Пантелеева Ирина Максимовна',
            'Пантелеева Ирина Максимовна', 'Глазунов Евгений Владимирович',
            'Глазунов Евгений Владимирович', 'Глазунов Евгений Владимирович', 
            'Глазунов Евгений Владимирович']
date_time = ['2018-02-11 09:00:00', '2018-02-11 10:30:00',
             '2018-02-11 12:10:00', '2018-02-11 09:00:00',
             '2018-02-11 10:30:00', '2018-11-19 14:10:00', 
             '2018-11-19 12:10:00']
classes = ['Компьютерная лингвистика и информационные технологии',
           'Компьютерная лингвистика и информационные технологии', 'Философия',
           'Философия', 'Базы данных', 'Программирование', 'Программирование']
emails = ['impanteleyeva@gmail.com', 'impanteleyeva@gmail.com', 
          'impanteleyeva@gmail.com', 'glaz.dikobraz@gmail.com', 
          'evgeniy_glaz@gmail.com', 'glaz.dikobraz@gmail.com', 
          'glaz.dikobraz@gmail.com']

In [9]:
for x in range(6):
    print('---')
    print(students[x], '\n', classes[x], '\n', teachers[x], '\n', batches[x],
          '\n', building_address[x], '\n', classrooms[x], '\n',
          date_time[x], '\n', emails[x], '\n')
    insert(mycursor, mydb,
           teacher_name=teachers[x],
           batch_name=batches[x],
           class_name=classes[x],
           building_address=building_address[x],
           classroom_num=classrooms[x],
           student_name=students[x],
           date_time=date_time[x],
           email=emails[x])

---
Пантелеева Ирина Максимовна 
 Компьютерная лингвистика и информационные технологии 
 Толдова Светлана Юрьевна 
 БКЛ-152 
 Старая Басманная ул., д. 21/4 
 505 
 2018-02-11 09:00:00 
 impanteleyeva@gmail.com 

Inserted.
---
Пантелеева Ирина Максимовна 
 Компьютерная лингвистика и информационные технологии 
 Нефедов Михаил Анатольевич 
 БКЛ-152 
 Кирпичная ул., д. 33 
 505 
 2018-02-11 10:30:00 
 impanteleyeva@gmail.com 

Inserted.
---
Пантелеева Ирина Максимовна 
 Философия 
 Золотухин Всеволод Валерьевич 
 БКЛ-152 
 Старая Басманная ул., д. 21/4 
 503 
 2018-02-11 12:10:00 
 impanteleyeva@gmail.com 

Inserted.
---
Глазунов Евгений Владимирович 
 Философия 
 Золотухин Всеволод Валерьевич 
 БКЛ-153 
 Старая Басманная ул., д. 21/4 
 502 
 2018-02-11 09:00:00 
 glaz.dikobraz@gmail.com 

Inserted.
---
Глазунов Евгений Владимирович 
 Базы данных 
 Клышинский Эдуард Станиславович 
 БКЛ-153 
 Старая Басманная ул., д. 21/4 
 321 
 2018-02-11 10:30:00 
 evgeniy_glaz@gmail.com 

Inserted.
---


Посмотрим как всё записалось:

In [10]:
tables = ['BATCHES', 'CLASSES', 'BUILDINGS', 'CLASSROOMS', 'STUDENTS', 'SCHEDULE']
for table in tables:
    mycursor.execute("SELECT * FROM %s" % table)
    myresult = mycursor.fetchall()
    print('---')
    print('Table:', table)
    for x in myresult:
        print(x)

---
Table: BATCHES
(1, 'БКЛ-152')
(2, 'БКЛ-153')
---
Table: CLASSES
(1, 'Компьютерная лингвистика и информационные технологии')
(2, 'Философия')
(3, 'Базы данных')
(4, 'Программирование')
---
Table: BUILDINGS
(1, 'Старая Басманная ул., д. 21/4')
(2, 'Кирпичная ул., д. 33')
---
Table: CLASSROOMS
(1, 505, 1)
(2, 505, 2)
(3, 503, 1)
(4, 502, 1)
(5, 321, 1)
---
Table: STUDENTS
(1, 'Пантелеева Ирина Максимовна', 1, 'impanteleyeva@gmail.com')
(2, 'Глазунов Евгений Владимирович', 2, 'glaz.dikobraz@gmail.com')
(3, 'Глазунов Евгений Владимирович', 2, 'evgeniy_glaz@gmail.com')
---
Table: SCHEDULE
(1, datetime.datetime(2018, 2, 11, 9, 0), 'Толдова Светлана Юрьевна', 1, 505, 1, 1, 1)
(2, datetime.datetime(2018, 2, 11, 10, 30), 'Нефедов Михаил Анатольевич', 1, 505, 2, 1, 1)
(3, datetime.datetime(2018, 2, 11, 12, 10), 'Золотухин Всеволод Валерьевич', 1, 503, 1, 2, 1)
(4, datetime.datetime(2018, 2, 11, 9, 0), 'Золотухин Всеволод Валерьевич', 2, 502, 1, 2, 2)
(5, datetime.datetime(2018, 2, 11, 10, 30)

### Запросы
    Задание: Вы должны показать владение операторами SELECT и прочими, показав что вы умеете делать JOIN, или искать пересечение таблиц в WHERE, или пользоваться вложенными SELECT.
    
    1) Показать расписание определённого студента
    2) Показать загруженность аудитории на определённый день
    3) Вывод всех предметов, корпусов, групп, кабинетов (для поиска - проще будет юзеру)
    4) Показать какие предметы идут в данную минуту
    5) Показать время, когда будет идти тот или иной предмет

#### 1) Показать расписание определённого студента

In [11]:
def schedule_of_student(email, mycursor):
    '''
    На вход: 'impanteleyeva@gmail.com'
    Возвращает: время, преподавателя, номер кабинета, предмет, группу 
    '''
    sql = "SELECT SCHEDULE.Date_Time, SCHEDULE.Teacher, \
                SCHEDULE.Classroom, BUILDINGS.Address, \
                CLASSES.Name, BATCHES.Name FROM SCHEDULE \
            JOIN BUILDINGS ON SCHEDULE.Building = BUILDINGS.id \
            JOIN CLASSES ON SCHEDULE.Class = CLASSES.id \
            JOIN BATCHES ON SCHEDULE.Batch = BATCHES.id \
            JOIN STUDENTS ON SCHEDULE.Student = STUDENTS.id \
            WHERE STUDENTS.Email = '%s'" % (email)
    mycursor.execute(sql)
    student = mycursor.fetchall()
    return student

In [12]:
student = schedule_of_student('impanteleyeva@gmail.com', mycursor)

In [13]:
student

[(datetime.datetime(2018, 2, 11, 9, 0),
  'Толдова Светлана Юрьевна',
  505,
  'Старая Басманная ул., д. 21/4',
  'Компьютерная лингвистика и информационные технологии',
  'БКЛ-152'),
 (datetime.datetime(2018, 2, 11, 10, 30),
  'Нефедов Михаил Анатольевич',
  505,
  'Кирпичная ул., д. 33',
  'Компьютерная лингвистика и информационные технологии',
  'БКЛ-152'),
 (datetime.datetime(2018, 2, 11, 12, 10),
  'Золотухин Всеволод Валерьевич',
  503,
  'Старая Басманная ул., д. 21/4',
  'Философия',
  'БКЛ-152')]

#### 2) Показать загруженность аудитории на определённый день

In [14]:
def classroom_load(classroom_num, day, building_address, mycursor):
    '''
    На вход: 505, '2018-02-11', 'Старая Басманная ул., д. 21/4'
    Возвращает: время (без дня), преподаватель
    '''
    sql = "SELECT EXTRACT(HOUR FROM SCHEDULE.Date_Time), \
            EXTRACT(MINUTE FROM SCHEDULE.Date_Time), \
            SCHEDULE.Teacher FROM SCHEDULE \
                JOIN BUILDINGS ON SCHEDULE.Building = BUILDINGS.id \
                WHERE BUILDINGS.Address = '%s' \
                    AND SCHEDULE.Classroom = %s \
                    AND MONTH(SCHEDULE.Date_Time) = MONTH('%s') \
                    AND DAY(SCHEDULE.Date_Time) = DAY('%s') \
                    AND YEAR(SCHEDULE.Date_Time) = YEAR('%s')" % (building_address, classroom_num,
                                                                  day, day, day)
    mycursor.execute(sql)
    load = mycursor.fetchall()
    return load

In [15]:
classroom_load(505, '2018-02-11', 'Старая Басманная ул., д. 21/4', mycursor)

[(9, 0, 'Толдова Светлана Юрьевна')]

#### 3) Вывод всех предметов, корпусов, групп, кабинетов (для поиска - проще будет юзеру)

In [16]:
def get_info(mycursor, classes=False, buildings=False, batches=False, classrooms=False):
    if classes:
        sql = "SELECT Name FROM CLASSES"
        mycursor.execute(sql)
        return mycursor.fetchall()
    if buildings:
        sql = "SELECT Address FROM BUILDINGS"
        mycursor.execute(sql)
        return mycursor.fetchall()
    if batches:
        sql = "SELECT Name FROM BATCHES"
        mycursor.execute(sql)
        return mycursor.fetchall()
    if classrooms:
        sql = "SELECT CLASSROOMS.Number, BUILDINGS.Address \
                FROM BUILDINGS JOIN CLASSROOMS ON BUILDINGS.id = CLASSROOMS.Address"
        mycursor.execute(sql)
        return mycursor.fetchall()
    else:
        return None

In [17]:
get_info(mycursor, classes=True)

[('Компьютерная лингвистика и информационные технологии',),
 ('Философия',),
 ('Базы данных',),
 ('Программирование',)]

In [18]:
get_info(mycursor, buildings=True)

[('Старая Басманная ул., д. 21/4',), ('Кирпичная ул., д. 33',)]

In [19]:
get_info(mycursor, batches=True)

[('БКЛ-152',), ('БКЛ-153',)]

In [20]:
get_info(mycursor, classrooms=True)

[(505, 'Старая Басманная ул., д. 21/4'),
 (505, 'Кирпичная ул., д. 33'),
 (503, 'Старая Басманная ул., д. 21/4'),
 (502, 'Старая Басманная ул., д. 21/4'),
 (321, 'Старая Басманная ул., д. 21/4')]

#### 4) Показать какие предметы идут в данную минуту

In [21]:
def right_now(mycursor):
    '''
    Возвращает: предметы
    '''
    sql = "SELECT SCHEDULE.Date_Time, CLASSES.Name \
            FROM CLASSES JOIN SCHEDULE ON CLASSES.id = SCHEDULE.Class \
            WHERE (SELECT NOW()>=SCHEDULE.Date_Time) \
            AND (SELECT NOW()<=DATE_ADD(SCHEDULE.Date_Time, INTERVAL '01:30' HOUR_MINUTE))"
    mycursor.execute(sql)
    return mycursor.fetchall()

In [22]:
right_now(mycursor)

[]

#### 5) Показать время, когда будет идти тот или иной предмет

In [23]:
def find_class(class_name, mycursor):
    '''
    Возвращает: предметы
    '''
    sql = "SELECT SCHEDULE.Date_Time, CLASSES.Name \
            FROM SCHEDULE JOIN CLASSES \
            ON SCHEDULE.Class = CLASSES.id WHERE CLASSES.Name = '%s'" % class_name
    mycursor.execute(sql)
    return mycursor.fetchall()

In [24]:
find_class('Программирование', mycursor)

[(datetime.datetime(2018, 11, 19, 14, 10), 'Программирование')]

### DELETE
    Пользователь вводит данные и они удаляются из базы данных.

In [25]:
def delete_batch(batch_name):
    '''
    Удаляет группу
    '''
    sql = "DELETE FROM STUDENTS WHERE Batch = (SELECT id FROM BATCHES WHERE BATCHES.Name = '%s')" % batch_name
    mycursor.execute(sql)
    sql = "DELETE FROM SCHEDULE WHERE Batch = (SELECT id FROM BATCHES WHERE BATCHES.Name = '%s')" % batch_name
    mycursor.execute(sql)
    sql = "DELETE FROM BATCHES WHERE Name = '%s'" % batch_name
    mycursor.execute(sql)
    mydb.commit()

In [26]:
delete_batch('БКЛ-152')

In [27]:
def delete_classes(class_name):
    '''
    Удаляет предмет
    '''
    sql = "DELETE FROM SCHEDULE WHERE Class = (SELECT id FROM CLASSES WHERE CLASSES.Name = '%s')" % class_name
    mycursor.execute(sql)
    sql = "DELETE FROM CLASSES WHERE Name = '%s'" % class_name
    mycursor.execute(sql)
    mydb.commit()

In [28]:
delete_classes('Философия')

In [29]:
def delete_buildings(building):
    '''
    Удаляет корпус
    '''
    sql = "DELETE FROM SCHEDULE WHERE Building = \
            (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s')" % building
    mycursor.execute(sql)
    sql = "DELETE FROM CLASSROOMS WHERE Address = \
            (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s')" % building
    mycursor.execute(sql)
    sql = "DELETE FROM BUILDINGS WHERE Address = '%s'" % building
    mycursor.execute(sql)
    mydb.commit()

In [30]:
delete_buildings('Кирпичная ул., д. 33')

In [31]:
def delete_classrooms_buildings(classroom, building):
    '''
    Удаляет кабинет (учитывая в каком здании он находится)
    '''
    sql = "DELETE FROM SCHEDULE WHERE Building = \
            (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s') \
            AND Classroom = %s" % (building, classroom)
    mycursor.execute(sql)
    sql = "DELETE FROM CLASSROOMS WHERE Address = \
            (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s') \
            AND Number = %s" % (building, classroom)
    mycursor.execute(sql)
    mydb.commit()

In [32]:
delete_classrooms_buildings(505, 'Кирпичная ул., д. 33')

In [33]:
def delete_student(email):
    '''
    Удаляет студента
    '''
    sql = "DELETE FROM SCHEDULE WHERE Student = (SELECT id FROM STUDENTS WHERE STUDENTS.Email = '%s')" % (email)
    mycursor.execute(sql)
    sql = "DELETE FROM STUDENTS WHERE Email = '%s'" % (email)
    mycursor.execute(sql)
    mydb.commit()

In [34]:
delete_student('evgeniy_glaz@gmail.com')

In [35]:
def delete_student_class(email, class_name, time, teacher, classroom, building):
    '''
    Удаляет определённый предмет в расписании
    '''
    sql = "DELETE FROM SCHEDULE WHERE Student = \
            (SELECT id FROM STUDENTS WHERE STUDENTS.Email = '%s') \
            AND Class = (SELECT id FROM CLASSES WHERE CLASSES.Name = '%s') \
            AND Date_time = '%s' AND Teacher = '%s' \
            AND Classroom = %s \
            AND Building = (SELECT id FROM BUILDINGS \
            WHERE BUILDINGS.Address = '%s')" % (email, class_name, time,
                                                teacher, classroom, building)
    mycursor.execute(sql)
    mydb.commit()

In [36]:
delete_student_class('impanteleyeva@gmail.com', 'Компьютерная лингвистика и информационные технологии', '2018-02-11 09:00:00', 'Толдова Светлана Юрьевна', 505, 'Старая Басманная ул., д. 21/4')

### UPDATE
    Изменить данные

In [37]:
def update_student_class(Nemail, Nclass_name, Ntime, Nteacher, Nclassroom, Nbuilding,
                         email, class_name, time, teacher, classroom, building):
    '''
    Удаляет определённый предмет в расписании
    '''
    sql = "UPDATE SCHEDULE \
            SET \
                Student = (SELECT id FROM STUDENTS WHERE STUDENTS.Email = '%s'), \
                Class = (SELECT id FROM CLASSES WHERE CLASSES.Name = '%s'), \
                Date_time = '%s', \
                Teacher = '%s', \
                Classroom = %s, \
                Building = (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s') \
            WHERE \
                Student = (SELECT id FROM STUDENTS WHERE STUDENTS.Email = '%s') AND \
                Class = (SELECT id FROM CLASSES WHERE CLASSES.Name = '%s') AND \
                Date_time = '%s' AND \
                Teacher = '%s' AND \
                Classroom = %s AND \
                Building = (SELECT id FROM BUILDINGS WHERE BUILDINGS.Address = '%s')" % (Nemail, Nclass_name,
                                                                                         Ntime, Nteacher, Nclassroom,
                                                                                         Nbuilding, email, class_name,
                                                                                         time, teacher, classroom, building)
    mycursor.execute(sql)
    mydb.commit()

In [38]:
update_student_class('glaz.dikobraz@gmail.com', 'Компьютерная лингвистика и информационные технологии',
                     '2018-02-11 09:00:00', 'Толдова Светлана Юрьевна', 505,
                     'Старая Басманная ул., д. 21/4', 'impanteleyeva@gmail.com',
                     'Компьютерная лингвистика и информационные технологии', '2018-02-11 09:00:00',
                     'Толдова Светлана Юрьевна', 505, 'Старая Басманная ул., д. 21/4')

In [39]:
for table in tables:
    mycursor.execute("SELECT * FROM %s" % table)
    myresult = mycursor.fetchall()
    print('---')
    print('Table:', table)
    for x in myresult:
        print(x)

---
Table: BATCHES
(2, 'БКЛ-153')
---
Table: CLASSES
(1, 'Компьютерная лингвистика и информационные технологии')
(3, 'Базы данных')
(4, 'Программирование')
---
Table: BUILDINGS
(1, 'Старая Басманная ул., д. 21/4')
---
Table: CLASSROOMS
(1, 505, 1)
(3, 503, 1)
(4, 502, 1)
(5, 321, 1)
---
Table: STUDENTS
(2, 'Глазунов Евгений Владимирович', 2, 'glaz.dikobraz@gmail.com')
---
Table: SCHEDULE
(6, datetime.datetime(2018, 11, 19, 14, 10), 'Литвинов Денис Владимирович', 2, 505, 1, 4, 2)
